# Brazil Weekly

## Connect with database and retrieve data

In [2]:
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import json
import time
import logging
import datetime
import udb

engine_ebdb = udb.get_ebdb_engine()
engine_analytics = udb.get_analytics_engine()
engine_airflow = udb.get_airflow_engine()

## Create tables

In [3]:
# Brazil database
users = pd.read_sql_table("auth_user", con=engine_ebdb)
subscriptions = pd.read_sql_table("payment_app_subscription", con=engine_ebdb)
company_orders = pd.read_sql_table("company_app_companyorder", con=engine_ebdb)
historical_subscriptions = pd.read_sql_table("payment_app_historicalsubscription", con=engine_ebdb)

## 1. Overall, how this week compares to the last?

In [5]:
day = datetime.date.today()
delta = 0 if day.weekday() == 6 else 7
dt = day #datetime.datetime.combine(day, datetime.time(23, 59, 59))
week_start = dt - datetime.timedelta(days=dt.weekday()) - datetime.timedelta(days=delta)
week_end = week_start + datetime.timedelta(days=7)

In [6]:
total_students_last_week = pd.read_sql_query("""SELECT count(DISTINCT payment_app_subscription.user_id) FROM payment_app_subscription
                                                INNER JOIN auth_user ON payment_app_subscription.user_id = auth_user.id
                                                WHERE status IN ('active','payment_credit_retry')
                                                  AND register_date::date <= '""" + week_start.strftime('%Y-%m-%d') + "'", con=engine_ebdb)['count'].iloc[0]

total_students_curr_week = pd.read_sql_query("""SELECT COUNT(DISTINCT payment_app_subscription.user_id) FROM payment_app_subscription
                                                INNER JOIN auth_user ON payment_app_subscription.user_id = auth_user.id
                                                WHERE status IN ('active','payment_credit_retry')
                                                  AND register_date::date <= '""" + week_end.strftime('%Y-%m-%d') + "'", con=engine_ebdb)['count'].iloc[0]

print("Brazil Paying students until " + week_start.strftime('%A, %b-%d') + ": " + str(total_students_last_week))
print("Brazil Paying students until " + week_end.strftime('%A, %b-%d') + ": " + str(total_students_curr_week))

Brazil Paying students until Monday, Nov-13: 7867
Brazil Paying students until Monday, Nov-20: 8127


In [57]:
paying_students_curr_week = pd.read_sql("""SELECT *
                                        FROM payment_app_subscription ps
                                        INNER JOIN payment_app_product pp ON ps.product_id = pp.id
                                        INNER JOIN auth_user au ON ps.user_id = au.id
                                        WHERE ps.status not IN ('invalid')
                                        and register_date::date between (current_date::date - cast(extract(isodow from current_date::date) 
                                        as int) + 1) 
                                        and (current_date::date - cast(extract(dow from  current_date::date) as int) + 7)""", con=engine_ebdb)

In [58]:
paying_students_last_week = pd.read_sql("""SELECT *
                                        FROM payment_app_subscription ps
                                        INNER JOIN payment_app_product pp ON ps.product_id = pp.id
                                        INNER JOIN auth_user au ON ps.user_id = au.id
                                        WHERE ps.status not IN ('invalid')
                                        and register_date::date between (current_date::date - cast(extract(isodow from current_date::date) 
                                        as int) - 6) 
                                        and (current_date::date - cast(extract(dow from  current_date::date) as int))""", con=engine_ebdb)

### Net additions last week

In [59]:
net_add = total_students_curr_week - total_students_last_week; 
print("Net additions last week: %d" % net_add)
perc_growth_vs_lw = net_add / total_students_last_week; 
print("Net growth vs last week: {0:+.1f}%".format(perc_growth_vs_lw * 100))

Net additions last week: 260
Net growth vs last week: +3.3%


### Paying students, cancellations, pauses and graduations

In [55]:
new_total_paying_students_curr_week = pd.read_sql_query("""SELECT COUNT(distinct ps.user_id) as new_std_this_week
                                        FROM payment_app_subscription ps
                                        INNER JOIN payment_app_product pp ON ps.product_id = pp.id
                                        INNER JOIN auth_user au ON ps.user_id = au.id
                                        WHERE ps.status not IN ('invalid')
                                        and register_date::date between (current_date::date - cast(extract(isodow from current_date::date) 
                                        as int) + 1) 
                                        and (current_date::date - cast(extract(dow from  current_date::date) as int) + 7)""", con=engine_ebdb)['new_std_this_week'].iloc[0]
print("New paying students this week: %d" % new_total_paying_students_curr_week)

New paying students this week: 119


In [56]:
new_total_paying_students_last_week = pd.read_sql_query("""SELECT COUNT(distinct ps.user_id) as new_std_this_week
                                        FROM payment_app_subscription ps
                                        INNER JOIN payment_app_product pp ON ps.product_id = pp.id
                                        INNER JOIN auth_user au ON ps.user_id = au.id
                                        WHERE ps.status not IN ('invalid')
                                        and register_date::date between (current_date::date - cast(extract(isodow from current_date::date) 
                                        as int) - 6) 
                                        and (current_date::date - cast(extract(dow from  current_date::date) as int))""", con=engine_ebdb)['new_std_this_week'].iloc[0]
print("New paying students last week: %d" % new_total_paying_students_last_week)

New paying students last week: 353


In [378]:
graduated_students_curr_week = pd.read_sql_query("""with grad_status_std AS
  (SELECT au.username, ps.id, ps.product_id, ps.graduation_date
   FROM payment_app_subscription ps
   INNER JOIN payment_app_product AS pp ON ps.product_id = pp.id
   INNER JOIN auth_user AS au ON ps.user_id = au.id
   WHERE ps.status IN ('graduated')), this_week AS
  (SELECT count(DISTINCT pg.id) AS grad_this_week FROM grad_status_std pg
   WHERE pg.graduation_date BETWEEN (CURRENT_DATE::date - cast(extract(isodow FROM CURRENT_DATE::date) AS int) + 1)
   AND (CURRENT_DATE::date - cast(extract(dow FROM CURRENT_DATE::date) AS int) + 7))
   SELECT * FROM this_week""", con=engine_ebdb)['grad_this_week'].iloc[0]
cancelled_students_curr_week = pd.read_sql_query("""with cancel_status_std AS
  (SELECT au.username, ps.id, ps.product_id, ps.cancel_date
   FROM payment_app_subscription ps
   INNER JOIN payment_app_product AS pp ON ps.product_id = pp.id
   INNER JOIN auth_user AS au ON ps.user_id = au.id
   WHERE ps.status IN ('cancelled')), this_week AS
  (SELECT count(DISTINCT pg.id) AS cancel_this_week FROM cancel_status_std pg
   WHERE pg.cancel_date BETWEEN (CURRENT_DATE::date - cast(extract(isodow FROM CURRENT_DATE::date) AS int) + 1)
   AND (CURRENT_DATE::date - cast(extract(dow FROM CURRENT_DATE::date) AS int) + 7))
   SELECT * FROM this_week""", con=engine_ebdb)['cancel_this_week'].iloc[0]
paused_students_curr_week = pd.read_sql_query("""with pause_status_std AS
  (SELECT au.username, ps.id, ps.product_id, ps.paused_date
   FROM payment_app_subscription ps
   INNER JOIN payment_app_product AS pp ON ps.product_id = pp.id
   INNER JOIN auth_user AS au ON ps.user_id = au.id
   WHERE ps.status IN ('paused')), 
   this_week AS
  (SELECT count(DISTINCT pg.id) AS paused_this_week FROM pause_status_std pg
   WHERE pg.paused_date BETWEEN (CURRENT_DATE::date - cast(extract(isodow FROM CURRENT_DATE::date) AS int) + 1)
   AND (CURRENT_DATE::date - cast(extract(dow FROM CURRENT_DATE::date) AS int) + 7))
   SELECT * FROM this_week""", con=engine_ebdb)['paused_this_week'].iloc[0]
drops_this_week = (cancelled_students_curr_week + paused_students_curr_week + graduated_students_curr_week)
"Cancellations, pauses and graduations this week: %d" % drops_this_week

'Cancellations, pauses and graduations this week: 23'

In [246]:
# Code to turnaround Nanodegree codes with ndXXX-XX
products = pd.read_sql_table("payment_app_product", con=engine_ebdb)
products['code2'] = products.apply(lambda row: row['code'].split('-')[0], axis=1)
products['code_old'] = products['code']
products['code'] = products['code2']
products

,id,name,code,price,sku,whitelist_enabled,acronym,accounting_sku,internal_name,access_period,access_model,access_unit,fifty_back_eligible,active,code2,code_old
0,19,Nanodegree Desenvolvedor de Realidade Virtual,nd017,399.0,6372143879290880,False,VRND,017-100,Nanodegree Desenvolvedor de Realidade Virtual ...,1,subscription,months,False,True,nd017,nd017
1,15,Nanodegree Test,nd800,1.0,6422863890087936,False,TestND,800-100,Nanodegree Test - Subscription - 1 months,1,subscription,months,False,True,nd800,nd800
2,21,Nanodegree Engenheiro de Inteligência Artificial,nd889,2400.0,4717113544671232,False,AIND,889-115,Nanodegree Engenheiro de Inteligência Artifici...,3,term_based,months,False,True,nd889,nd889
3,14,Nanodegree Introdução à Programação,nd000,399.0,4723990152871936,False,IPND,000-100,Nanodegree Introdução à Programação - Subscrip...,1,subscription,months,False,True,nd000,nd000
4,13,Nanodegree Desenvolvedor Web Front-End,nd001,399.0,6087356385591296,False,FEND,001-100,Nanodegree Desenvolvedor Web Front-End - Subsc...,1,subscription,months,False,True,nd001,nd001
5,12,Nanodegree Desenvolvedor iOS,nd003,399.0,5358809337561088,False,IOSND,003-100,Nanodegree Desenvolvedor iOS - Subscription - ...,1,subscription,months,False,True,nd003,nd003
6,20,Nanodegree Predictive Analytics,nd008,399.0,5539181600309248,False,PAND,008-100,Nanodegree Predictive Analytics - Subscription...,1,subscription,months,False,True,nd008,nd008
7,23,Deep Learning Nanodegree Foundation,nd101,999.0,5976198699155456,False,DLNDF,101-130,Deep Learning Nanodegree Foundation - Term-bas...,6,term_based,months,False,True,nd101,nd101
8,11,Nanodegree Desenvolvedor Full-Stack,nd004,399.0,5229920858669056,False,FSND,004-100,Nanodegree Desenvolvedor Full-Stack - Subscrip...,1,subscription,months,False,True,nd004,nd004
9,10,Nanodegree Iniciando com iOS,nd006,399.0,6367474689507328,False,BiOSND,006-100,Nanodegree Iniciando com iOS - Subscription - ...,1,subscription,months,False,True,nd006,nd006


In [266]:
english_names = pd.read_sql_table('nd_names_in_english', con=engine_airflow)[['code', 'name']]
english_names.rename(columns={'name': 'eng_name1'}, inplace=True)
products_eng_names = products.join(english_names.set_index('code'), on='code')
products_eng_names = products_eng_names[['code2', 'eng_name1']]
products_eng_names = products_eng_names.drop_duplicates()
products_eng_names

,code2,eng_name1
0,nd017,Virtual Reality
1,nd800,Test
2,nd889,Artificial Intelligence
3,nd000,Intro to Programming
4,nd001,Front-end Developer
5,nd003,iOS Developer
6,nd008,Predictive Analytics
7,nd101,Deep Learning
8,nd004,Full-stack Developer
9,nd006,Beginning iOS


In [267]:
paying_students_curr_week_grouped = paying_students_curr_week[['user_id','code']].groupby(['code']).agg(['count'])
paying_students_curr_week_grouped = paying_students_curr_week_grouped.reset_index()

In [345]:
count_code_curr_week = paying_students_curr_week_grouped.copy()
count_code_curr_week.columns = count_code_curr_week.columns.droplevel()
count_code_curr_week.columns=['code', 'count']
count_code_curr_week = count_code_curr_week.sort_values('count', ascending=False)
count_code_curr_week['code'] = paying_students_curr_week_by_nd.apply(lambda row: row['code'].split('-')[0], axis=1)
count_code_curr_week.reset_index(inplace=True,drop=True)

In [356]:
top_nds = count_code_curr_week
top_nds = count_code_curr_week.join(products_eng_names.set_index('code2'), on='code')
top_nds = top_nds.rename(columns={'eng_name1': 'product_id', 'count': 'students'})
top_nds = top_nds[['product_id','students']]
top_nds = top_nds.fillna('empty')
top_nds['%'] = top_nds['students'] / new_total_paying_students_curr_week
top_nds

,product_id,students,%
0,Data Science Fundamentals I,36,0.302521
1,Digital Marketing,17,0.142857
2,Data Science Fundamentals II,13,0.109244
3,Machine Learning,11,0.092437
4,React Developer,8,0.067227
5,Front-end Developer,7,0.058824
6,Predictive Analytics,6,0.050420
7,Intro to Programming,5,0.042017
8,React Developer,5,0.042017
9,Data Science for Business,4,0.033613


In [373]:
cum = 0
last_index = 0
top_nds['cum'] = 0
for index, row in top_nds.iterrows():
    cum += row['%']
    top_nds['cum'].loc[index] = cum
top_nds = top_nds[top_nds['cum'] < 0.9]
top_nds

/Users/udacity-nathaliapaula/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,product_id,students,%,cum
0,Data Science Fundamentals I,36,0.302521,0.302521
1,Digital Marketing,17,0.142857,0.445378
2,Data Science Fundamentals II,13,0.109244,0.554622
3,Machine Learning,11,0.092437,0.647059
4,React Developer,8,0.067227,0.714286
5,Front-end Developer,7,0.058824,0.773109
6,Predictive Analytics,6,0.050420,0.823529
7,Intro to Programming,5,0.042017,0.865546


### Total graduations

In [369]:
grads_last_week = pd.read_sql_query("""SELECT count(DISTINCT payment_app_subscription.user_id) FROM payment_app_subscription
                                                INNER JOIN auth_user ON payment_app_subscription.user_id = auth_user.id
                                                WHERE status IN ('graduated')
                                                  AND graduation_date::date <= '""" + week_start.strftime('%Y-%m-%d') + "'", con=engine_ebdb)['count'].iloc[0]

grads_curr_week = pd.read_sql_query("""SELECT COUNT(DISTINCT payment_app_subscription.user_id) FROM payment_app_subscription
                                                INNER JOIN auth_user ON payment_app_subscription.user_id = auth_user.id
                                                WHERE status IN ('graduated')
                                                  AND graduation_date::date <= '""" + week_end.strftime('%Y-%m-%d') + "'", con=engine_ebdb)['count'].iloc[0]



print("Brazil graduates until " + week_start.strftime('%A, %b-%d') + ": " + str(grads_last_week))
print("Brazil graduates until " + week_end.strftime('%A, %b-%d') + ": " + str(grads_curr_week))

Brazil graduates until Monday, Nov-13: 1362
Brazil graduates until Monday, Nov-20: 1424


In [374]:
net_grad = grads_curr_week - grads_last_week; 
print("Net graduations last week: %d" % net_grad)
perc_grad_vs_lw = net_grad / grads_last_week; 
print("Net graduations vs last week: {0:+.1f}%".format(perc_grad_vs_lw * 100))

Net graduations last week: 62
Net graduations vs last week: +4.6%


### Opening paragraph

In [375]:
from IPython.display import Markdown, display
import locale

locale.setlocale(locale.LC_ALL, 'en_US')

def printmd(string):
    display(Markdown(string))

In [386]:
# First sentence

if graduated_students_curr_week == 0:
    grad_str = ''
elif graduated_students_curr_week == 1:
    grad_str = ' and **1 graduation**'
else:
    grad_str = ' and **%d graduations**' % graduated_students_curr_week

# Second sentence
main_nds_str = 'This was mainly driven by '

for index, row in top_nds.iterrows():
    main_nds_str += row['product_id'] + " **(+" + str(row['students']) + ")**"
    if (index + 1) == top_nds.shape[0]:
        main_nds_str += '.'
    elif (index + 2) == top_nds.shape[0]:
        main_nds_str += ' and '
    else:
        main_nds_str += ', '
    
printmd(
    """#### Summary
We closed this week with **%s** paying students, **%s** vs. last week, 
**%s** new paying students partially offset by **%s** cancellations/pauses%s. 
%s
""" 
    % (locale.format("%d", total_students_curr_week, grouping=True),
       "{0:+.1f}%".format(perc_growth_vs_lw * 100),
       locale.format("%+d", new_paying_students_curr_week, grouping=True),
       locale.format("%d", (cancelled_students_curr_week + paused_students_curr_week) , grouping=True),
       grad_str,
       main_nds_str
      )
)

# Table
str_1 = locale.format("%d", total_students_curr_week, grouping=True)
str_2 = "{0:+.1f}%".format(perc_growth_vs_lw * 100)
str_3 = ''
str_4 = "{:,}".format(grads_curr_week)
str_5 = "{0:+.1f}%".format(perc_grad_vs_lw * 100)

table_str = """
Q3 2017 Key Results | Last week
--- | ---
**KR1:** 10k paying students (10.5k stretch) | """ + str_1 + """ (""" + str_2 + """)
**KR2:** 1,500 graduates (1,800 stretch)  | """ + str_4 + """ (""" + str_5 + """)
**KR3:** P2 pass rate from 24% to 30% | """+ str_3 + """
"""
printmd(table_str)

#### Summary
We closed this week with **8,127** paying students, **+3.3%** vs. last week, 
**+118** new paying students partially offset by **13** cancellations/pauses and **10 graduations**. 
This was mainly driven by Data Science Fundamentals I **(+36)**, Digital Marketing **(+17)**, Data Science Fundamentals II **(+13)**, Machine Learning **(+11)**, React Developer **(+8)**, Front-end Developer **(+7)**, Predictive Analytics **(+6)** and Intro to Programming **(+5)**.



Q3 2017 Key Results | Last week
--- | ---
**KR1:** 10k paying students (10.5k stretch) | 8,127 (+3.3%)
**KR2:** 1,500 graduates (1,800 stretch)  | 1,424 (+4.6%)
**KR3:** P2 pass rate from 24% to 30% | 
